# 🧪 Interface Gradio para Explorar Logs

In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import gradio as gr
from sklearn.ensemble import IsolationForest

# 📥 Carregar dados de compras de ingressos e aplicar IA
def carregar_dados_ingressos_com_ia():
    conn = sqlite3.connect("../database.db")
    query = """
        SELECT u.id AS usuario_id, u.nome, u.email, l.acao_usuario as acao, l.data_hora
        FROM logs l
        JOIN usuarios u ON u.id = l.usuario_id
        WHERE l.acao_usuario LIKE 'Compra do ingresso ID%'
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['evento'] = df['acao'].str.replace("Compra do ingresso ID ", "Ingresso ")
    df['data'] = df['data_hora'].dt.date

    # 🧠 IA: Isolation Forest
    df_features = df.groupby(['usuario_id', 'nome', 'email']).agg(
        total_compras=('evento', 'count'),
        dias_diferentes=('data', pd.Series.nunique)
    ).reset_index()

    modelo = IsolationForest(contamination=0.15, random_state=42)
    df_features['classificacao'] = modelo.fit_predict(df_features[['total_compras', 'dias_diferentes']])
    df_features['classificacao'] = df_features['classificacao'].map({1: 'Normal', -1: 'Anômalo'})

    # Junta classificação de volta ao dataset original
    df = pd.merge(df, df_features[['usuario_id', 'classificacao']], on='usuario_id', how='left')

    return df

# 🎯 Gerar gráficos com filtros
def gerar_dashboard(email="", nome="", usuario_id=""):
    df = carregar_dados_ingressos_com_ia()

    if email:
        df = df[df['email'].str.contains(email, case=False, na=False)]
    if nome:
        df = df[df['nome'].str.contains(nome, case=False, na=False)]
    if usuario_id:
        try:
            usuario_id = int(usuario_id)
            df = df[df['usuario_id'] == usuario_id]
        except:
            return "⚠️ ID inválido.", None, None, None, None

    if df.empty:
        return "⚠️ Nenhuma compra encontrada com os filtros aplicados.", None, None, None, None

    # 🎫 Compras por evento
    evento_count = df['evento'].value_counts().reset_index()
    evento_count.columns = ['Evento', 'Quantidade']
    fig1 = px.bar(evento_count, x='Evento', y='Quantidade', title='🎫 Compras por Evento')

    # 👤 Compras por usuário
    user_count = df.groupby(['nome', 'classificacao'])['evento'].count().reset_index(name='Compras')
    fig2 = px.bar(user_count, x='nome', y='Compras', color='classificacao',
                  title='👤 Compras por Usuário (com Classificação)')

    # 📅 Evolução por data
    data_count = df['data'].value_counts().sort_index().reset_index()
    data_count.columns = ['Data', 'Quantidade']
    data_count['Data'] = data_count['Data'].apply(lambda d: d.strftime('%d/%m/%Y'))
    fig3 = px.line(data_count, x='Data', y='Quantidade', markers=True, title='📆 Compras por Data')

    # 🥧 Pizza por classificação
    class_count = df['classificacao'].value_counts().reset_index()
    class_count.columns = ['Classificação', 'Quantidade']
    fig4 = px.pie(class_count, names='Classificação', values='Quantidade',
                  title='🥧 Distribuição das Classificações (IA)')

    return "✅ Dados carregados com sucesso!", fig1, fig2, fig3, fig4

# 🌐 Interface Gradio alternativa com gr.Interface
interface = gr.Interface(
    fn=gerar_dashboard,
    inputs=[
        gr.Textbox(label="📩 Email"),
        gr.Textbox(label="👤 Nome do Usuário"),
        gr.Textbox(label="🆔 ID do Usuário")
    ],
    outputs=[
        gr.Textbox(label="Status"),
        gr.Plot(label="🎫 Compras por Evento"),
        gr.Plot(label="👤 Compras por Usuário"),
        gr.Plot(label="📆 Compras por Data"),
        gr.Plot(label="🥧 Classificação por IA")
    ],
    title="🎟️ Dashboard de Compras de Ingressos com Detecção de Anomalias (IA)",
    description="Use os filtros para analisar o comportamento de compra e identificar possíveis anomalias com Isolation Forest."
)

# 🚀 Lançar app
if __name__ == "__main__":
    interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
